<a href="https://colab.research.google.com/github/gauri131202/JPMC_Final_Project/blob/main/case_study_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#connecting to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Importing all the necessary libraries

import cv2
import numpy as np
import os
import matplotlib.pyplot as plt


Cleaning and preprocessing the images

In [ ]:
#cleaning the image
# Function to apply thresholding and opening to an image
def preprocess_image(image):

    # Apply OTSU thresholding
    _, thresholded = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)


    # Define a kernel for the opening operation
    kernel = np.ones((2, 2), np.uint8)


    opened = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel)

    return opened

#Path to the directory containing the image dataset
dataset_dir = "/content/drive/MyDrive/JPMC/samples"

preprocessed_array= []

# Loop through the images in the dataset directory
for filename in os.listdir(dataset_dir):
    if filename.endswith(".png"):
        # Read the image
        image_path = os.path.join(dataset_dir, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Preprocess the image
        preprocessed_image = preprocess_image(image)

        # Save the preprocessed image
        preprocessed_array.append(preprocessed_image)

print(len(preprocessed_array))

1000


In [ ]:
#extracting the file names to train the models

filename_characters = []
filenames = []

for file in os.listdir(dataset_dir):
  filename, extension = os.path.splitext(file)
  filenames.append(filename)

for names in filenames:
  string = names

  # Convert the string to a list of characters
  characters = list(string)

  for character in characters:
    filename_characters.append(character)

len(filename_characters)

5000

Dividing each image into 5 parts of 100X20 pixels

In [ ]:
#cropping the image into 5 parts

cropped_array = []
for i in range(len(preprocessed_array)):
  img = preprocessed_array[i]

  width = 20

    # Iterate over the width of the image to extract character images
  for x in range(5):
        # Calculate the starting and ending coordinates for each character
      start = x * width
      end = start + width

        # Extract the character image
      cropped = img[0:100, start:end]
      #character_image = img.crop((start_x, 0, end_x, height))

      cropped_array.append(cropped)

print(len(cropped_array))

5000


# Model 1: Logistic Regression

In [ ]:
#Importing the required libraries
import scipy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
#Loading the dataset
X = np.array(cropped_array)
y = np.array(filename_characters)

#Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=45)

#Reshaping the images
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

#Training the model
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

#Running on the test set
y_pred = classifier.predict(X_test)

#Evaluating the Logistic Regression model
accuracy = classifier.score(X_test, y_test)
print("Accuracy: ", accuracy)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


Accuracy:  0.477


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.477

# Model 2: Support Vector Machines

In [ ]:
#Importing the required libraries
from sklearn import svm
from sklearn.model_selection import train_test_split

In [ ]:
#Loading the dataset
X = np.array(cropped_array)
Y = np.array(filename_characters)

#Splitting the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Reshaping the images
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

#Creating an SVM classifier
classifier = svm.SVC()

#Training the SVM classifier
classifier.fit(X_train, Y_train)

#Predicting the labels for the test set
Y_pred = classifier.predict(X_test)

#Evaluating the accuracy of the classifier
accuracy = classifier.score(X_test, Y_test)
print("Accuracy:", accuracy)

Accuracy: 0.653


# Model 3: Random Forest

In [ ]:
#Importing the required libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
#Loading the dataset
X = np.array(cropped_array)
Y = np.array(filename_characters)

#Splitting the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Reshaping the images
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

#Creating a Random Forest classifier
classifier = RandomForestClassifier()

#Training the Random Forest classifier
classifier.fit(X_train, Y_train)

#Predicting the labels for the test set
Y_pred = classifier.predict(X_test)

#Evaluating the accuracy of the classifier
accuracy = classifier.score(X_test, Y_test)
print("Accuracy:", accuracy)

Accuracy: 0.672


In [ ]:
#Hyperparameter tuning
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Loading the dataset
X = np.array(cropped_array)
Y = np.array(filename_characters)

# Splitting the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Reshaping the images
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Creating a Random Forest classifier
classifier = RandomForestClassifier(n_estimators=600,max_depth=30,min_samples_split=5 ,min_samples_leaf=1,max_features='sqrt')

# Training the Random Forest classifier
classifier.fit(X_train, Y_train)

# Predicting the labels for the test set
Y_pred = classifier.predict(X_test)

# Evaluating the accuracy of the classifier
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.718


## **Answer to the questions mentioned:**

1. Which of the above classifiers yield the best accuracy on the validation set? Do you think accuracy is the best metric to compare the algorithms, if not - which other metric could be used?

  The Random Forest classifier yielded the best accuracy- 0.678 -on the validation set.
  
  No, accuracy is not always the best metric to compare models.
  Here are a few commonly used metrics:
  1. **Precision**: Precision measures the proportion of correctly predicted positive instances (true positives) out of the total instances predicted as positive (true positives + false positives). It focuses on the model's ability to avoid false positives.
  2. **Recall** (Sensitivity or True Positive Rate): Recall calculates the proportion of correctly predicted positive instances (true positives) out of the total actual positive instances (true positives + false negatives). It emphasizes the model's ability to identify all positive instances.
  3. **F1 Score**: The F1 score is the harmonic mean of precision and recall. It provides a balanced measure that considers both precision and recall. It is useful when there is an uneven distribution of classes or when you want to find a balance between precision and recall.
  4. **Area Under the ROC Curve** (AUC-ROC): AUC-ROC measures the performance of a binary classifier at various classification thresholds. It considers the true positive rate (sensitivity) against the false positive rate (1 - specificity) across different thresholds. It provides an aggregate measure of the model's performance across all possible thresholds.
  5. **Confusion Matrix**: A confusion matrix is a tabular representation that shows the counts of true positives, true negatives, false positives, and false negatives. It provides a detailed breakdown of the model's predictions and can be used to calculate metrics like accuracy, precision, recall, and F1 score.

2. Tweak the hyperparameters of the classifier selected in Q1 and try to improve the accuracy. What hyperparameters did you change, why do you think it worked?

  <Answer> To tweak the hyperparameters of the Random Forest classifier and potentially improve the accuracy, the following parameters can be modified:

  1.   **n_estimators**: The number of decision trees being built in the forest. Default values in sklearn are 100. N_estimators are mostly correlated to the size of data, to encapsulate the trends in the data, more number of DTs are needed.

  2.   **max_depth**: It controls the maximum depth of each decision tree in the forest. Increasing max_depth allows the trees to capture more complex relationships in the data but also increases the risk of overfitting.

  3. **min_samples_split**: This parameter determines the minimum number of samples required to split an internal node. Increasing this value can help prevent overfitting by requiring more samples for a split.

  4. **min_samples_leaf**: It sets the minimum number of samples required to be at a leaf node. Similar to min_samples_split, increasing min_samples_leaf can prevent overfitting by enforcing a minimum number of samples in each leaf.

  5. **max_features**: This parameter controls the number of features to consider when looking for the best split. Reducing the number of features can help prevent overfitting, especially if the dataset has many features.

After playing around with the parameter values, and using trial and error methods, we came up with the most optimised values as follows:

`classifier = RandomForestClassifier(n_estimators=600,max_depth=30,min_samples_split=5 ,min_samples_leaf=1,max_features='sqrt')`

This improved the accuracy of the model to 0.718


3. Find Precision, Recall and F1 score in all cases.

  <Answer>
  1. **Random Forest:**
    * Precision: 0.7449812348667282
    * Recall: 0.7153426940633367
    * F1 Score: 0.7203288485030938

  2. **Support Vector Machine:**
    * Precision: 0.6796350234338434
    * Recall: 0.6492042178366149
    * F1 Score: 0.6547169511226476
  
  3. **Logistic Regression:**
    * Precision: 0.5920632023311434
    * Recall: 0.5582204276332229
    * F1 Score: 0.5767232124510447



  





4. a. Find the confusion matrix using the best classifier.
  
  b. Which characters have low accuracy?
  
  c. What can be the reason for this?

  <Answer>

  a. The confusion matrix can be found using the code
`confusion_mat = confusion_matrix(Y_test, Y_pred)`, and importing the library   confusion_matrix from sklearn.metrics

The code is as follows:


In [ ]:
#Importing the required libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

X = np.array(cropped_array)
Y = np.array(filename_characters)

#Splitting the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Reshaping the images
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

#Creating a Random Forest classifier
classifier = RandomForestClassifier()

#Training the Random Forest classifier
classifier.fit(X_train, Y_train)

#Predicting the labels for the test set
Y_pred = classifier.predict(X_test)
confusion_mat = confusion_matrix(Y_test, Y_pred)

# Printing the confusion matrix
print("Confusion Matrix:")
print(confusion_mat)

Confusion Matrix:
[[38  0  0  0  0  0  0  0  2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  2]
 [ 1 31  0  1  0  1  0  2  2  0  1  2  0  0  0  0  0  0  0  1  0  0  0  0
   0  1]
 [ 0  0 31  0  0  1  0  0  1  1  0  2  0  0  1  0  0  1  0  0  0  0  0  0
   0  0]
 [ 0  2  0 25  0  3  0  0  1  1  0  0  1  0  0  0  0  0  0  1  0  0  1  0
   0  1]
 [ 5  0  0  0 28  0  0  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0  0
   0  1]
 [ 0  0  0  0  0 24  0  0  2  4  1  0  0  0  0  0  0  0  0  0  0  0  0  0
   2  1]
 [ 1  1  0  0  1  0 21  0  0  2  0  0  0  0  0  0  0  0  1  1  0  0  0  1
   0  1]
 [ 0  9  0  3  0  4  0 20  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0  1
   0  0]
 [ 0  0  0  0  0  4  0  0 16  1  0  3  0  0  0  0  0  0  0  0  0  0  0  3
   0  0]
 [ 0  1  0  0  0  1  0  0  0 42  0  1  0  0  0  1  0  0  0  0  0  1  0  0
   0  0]
 [ 0  0  1  1  0  0  0  0  2  0 31  1  0  0  0  0  0  0  0  1  0  3  0  6
   1  0]
 [ 0  0  0  0  0  3  0  0  5  5  0 24  0  0  0  0  0  1  0  0  0  0  

b. We found the accuracy for each character by dividing the values on the diagonal with the total number of samples per character. This gave us the accuracy for each character. The code is as follows:

In [ ]:
#Importing the required libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

X = np.array(cropped_array)
Y = np.array(filename_characters)

#Splitting the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Reshaping the images
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))

#Creating a Random Forest classifier
classifier = RandomForestClassifier()

#Training the Random Forest classifier
classifier.fit(X_train, Y_train)

#Predicting the labels for the test set
Y_pred = classifier.predict(X_test)
confusion_mat = confusion_matrix(Y_test, Y_pred)


def calculate_accuracy(confusion_mat, character_index):
    successes = confusion_mat[character_index, character_index]
    total_samples = np.sum(confusion_mat[character_index, :])
    accuracy = successes / total_samples
    return accuracy

# Calculate accuracy for each character
num_characters = confusion_mat.shape[0]
accuracies = []
for i in range(num_characters):
    accuracy = calculate_accuracy(confusion_mat, i)
    accuracies.append(accuracy)

# Print accuracies
for i, accuracy in enumerate(accuracies):
    print(f"Character {i}: {accuracy*100:.2f}%")

Character 0: 88.37%
Character 1: 65.12%
Character 2: 84.21%
Character 3: 69.44%
Character 4: 72.22%
Character 5: 67.65%
Character 6: 73.33%
Character 7: 43.59%
Character 8: 59.26%
Character 9: 91.49%
Character 10: 59.57%
Character 11: 67.50%
Character 12: 51.35%
Character 13: 54.35%
Character 14: 71.43%
Character 15: 65.71%
Character 16: 73.33%
Character 17: 67.86%
Character 18: 60.98%
Character 19: 57.50%
Character 20: 61.36%
Character 21: 63.41%
Character 22: 84.85%
Character 23: 60.53%
Character 24: 51.16%
Character 25: 63.27%


This clearly shows that character 7,8,11,13,19,and 23, i.e '**h', 'i', 'l', 'n', 't', and 'x'** have low accuracy

c. Few of the reasons for low acccuracy of these alphabets could be:

1. Class imbalance: If the dataset has imbalanced class distributions, with some characters having significantly fewer samples compared to others, the classifier may be biased towards the majority classes, resulting in lower accuracy for the minority classes.
2. Variability in writing styles or fonts: If the dataset contains characters written in different styles or fonts, it can introduce additional variations and challenges for the classifier to generalize well, resulting in lower accuracy.
3. Insufficient training data: If there is limited training data available for certain characters, the classifier may not have enough examples to learn their distinguishing features effectively, leading to lower accuracy.
4. Feature representation: The chosen features for character representation may not capture the essential characteristics that differentiate certain characters effectively, leading to lower accuracy. Improving the feature representation or using more advanced feature extraction techniques can potentially address this issue.

5. Try any other techniques/algorithms in your research that could improve the accuracy.

  One way to improve the accuracy is applying Convolutional Nueral Network(CNN) to the dataset.


In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Assuming 'cropped_array' is your input data with dimensions 4000x2000
X = np.array(cropped_array)
Y = np.array(filename_characters)

# Use LabelEncoder to encode character labels into numeric values
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

# Splitting the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_encoded, test_size=0.2, random_state=42)

# Define the dimensions of each sample
sample_width = 32
sample_height = 32

# Resize and reshape the training set
X_train_resized = []
for img in X_train:
    # Resize the image to (32, 32)
    resized_img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
    # Reshape the grayscale image to (32, 32, 1)
    reshaped_img = np.reshape(resized_img, (32, 32, 1))
    X_train_resized.append(reshaped_img)
X_train_resized = np.array(X_train_resized)

# Resize and reshape the test set
X_test_resized = []
for img in X_test:
    # Resize the image to (32, 32)
    resized_img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
    # Reshape the grayscale image to (32, 32, 1)
    reshaped_img = np.reshape(resized_img, (32, 32, 1))
    X_test_resized.append(reshaped_img)
X_test_resized = np.array(X_test_resized)

# Create a CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_resized, to_categorical(Y_train), batch_size=32, epochs=50, verbose=1, validation_split=0.2)

# Evaluate the model
accuracy = model.evaluate(X_test_resized, to_categorical(Y_test))[1]
print("Accuracy:", accuracy)


Epoch 1/50
100/100 [==============================] - 6s 53ms/step - loss: 52.5272 - accuracy: 0.0584 - val_loss: 3.2272 - val_accuracy: 0.0437
Epoch 2/50
100/100 [==============================] - 3s 31ms/step - loss: 3.1318 - accuracy: 0.0669 - val_loss: 3.1334 - val_accuracy: 0.0538
Epoch 3/50
100/100 [==============================] - 3s 29ms/step - loss: 2.9618 - accuracy: 0.1241 - val_loss: 3.0455 - val_accuracy: 0.0925
Epoch 4/50
100/100 [==============================] - 3s 29ms/step - loss: 2.7275 - accuracy: 0.1966 - val_loss: 2.9304 - val_accuracy: 0.1538
Epoch 5/50
100/100 [==============================] - 5s 47ms/step - loss: 2.4995 - accuracy: 0.2466 - val_loss: 2.7777 - val_accuracy: 0.2013
Epoch 6/50
100/100 [==============================] - 3s 34ms/step - loss: 2.3079 - accuracy: 0.2916 - val_loss: 2.7260 - val_accuracy: 0.2275
Epoch 7/50
100/100 [==============================] - 3s 31ms/step - loss: 2.1212 - accuracy: 0.3375 - val_loss: 2.5965 - val_accuracy: 0.242